In [5]:
import sys
import subprocess
import pkg_resources
import os
import pandas as pd
sys.path.append(os.path.abspath('../scripts'))

from myFunctions import install_packages, save_table 
install_packages()


numpy is already installed.
pandas is already installed.
scikit-learn is already installed.
joblib is already installed.
pyarrow is already installed.
fastparquet is already installed.
plotly is already installed.
matplotlib is already installed.
MetaTrader5 is already installed.
All packages are verified


In [6]:
input_target = '..//data//target//'
input_features = '..//data//features//'
output_dir = '..//data//processed_data//'

In [51]:
df_features = pd.read_parquet(f'{input_features}features.parquet')
df_daily_target =  pd.read_parquet(f'{input_target}daily_target.parquet')
df_timestamp_target =  pd.read_parquet(f'{input_target}timestamp_target.parquet')
df_daily = df_features.copy()
df_timestamp = df_features.copy()



In [52]:
print(df_timestamp.shape)
print(df_timestamp_target.shape)
df_timestamp = df_timestamp.merge(
    df_timestamp_target[['time', 'close_price_target', 'open_price_target', 'behavior_target']],
    on='time',
    how='inner'
)
print(df_timestamp.shape)

(22356, 85)
(22356, 6)
(22356, 88)


In [53]:
df_timestamp_target.head()

,time,open_BGI$,close_BGI$,close_price_target,open_price_target,behavior_target
96,2022-06-02 09:00:00,313.94,313.75,312.73,313.17,0
97,2022-06-02 09:15:00,313.17,312.73,312.73,313.17,0
98,2022-06-02 09:30:00,313.17,312.73,312.78,313.41,1
99,2022-06-02 09:45:00,313.41,312.78,313.56,312.83,1
100,2022-06-02 10:00:00,312.83,313.56,313.94,313.89,1


In [64]:
df_timestamp

,time,open_BGI$,high_BGI$,low_BGI$,close_BGI$,tick_volume_BGI$,spread_BGI$,real_volume_BGI$,open_CCM$,high_CCM$,...,EMA55_GOLD11,EMA9_BGI$,EMA21_BGI$,EMA55_BGI$,EMA9_IBOV,EMA21_IBOV,EMA55_IBOV,close_price_target,open_price_target,behavior_target
0,2022-06-02 09:00:00,313.94,314.62,313.07,313.75,44.0,1.0,14.0,121.02,122.52,...,9.380451,312.964032,313.169828,313.822989,111438.765770,111480.083554,111455.854537,312.73,313.17,0
1,2022-06-02 09:15:00,313.17,313.17,312.73,312.73,4.0,1.0,4.0,122.10,122.76,...,9.382935,312.917225,313.129843,313.783954,111423.012616,111469.166867,111452.431161,312.73,313.17,0
2,2022-06-02 09:30:00,313.17,313.17,312.73,312.73,0.0,1.0,0.0,122.45,122.66,...,9.385330,312.879780,313.093494,313.746313,111410.410093,111459.242606,111449.130048,312.78,313.41,1
3,2022-06-02 09:45:00,313.41,313.41,312.78,312.78,5.0,1.0,5.0,121.98,121.98,...,9.387640,312.859824,313.064995,313.711802,111400.328074,111450.220551,111445.946832,313.56,312.83,1
4,2022-06-02 10:00:00,312.83,313.99,312.83,313.56,9.0,1.0,9.0,121.64,121.78,...,9.389510,312.999859,313.109995,313.706380,111577.062459,111526.018683,111475.877302,313.94,313.89,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22351,2024-11-22 16:45:00,349.45,350.00,348.75,350.00,0.0,0.0,0.0,71.97,72.14,...,16.385959,349.567639,348.769420,346.888650,128578.593827,128362.972960,127974.196554,349.45,349.45,0
22352,2024-11-22 17:00:00,349.45,349.45,349.45,349.45,2.0,0.0,3.0,72.11,72.28,...,16.390389,349.544111,348.831291,346.980127,128564.475061,128376.157237,127993.260963,350.00,349.85,1
22353,2024-11-22 17:15:00,349.85,350.00,349.85,350.00,13.0,0.0,80.0,72.05,72.05,...,16.394303,349.635289,348.937537,347.087980,128567.180049,128394.506579,128014.144500,350.00,350.00,0
22354,2024-11-22 17:30:00,350.00,350.00,350.00,350.00,6.0,0.0,102.0,72.08,72.20,...,16.397721,349.708231,349.034125,347.191980,128627.944039,128437.824162,128044.746482,350.50,350.00,1


In [55]:
df_daily.head()

,time,open_BGI$,high_BGI$,low_BGI$,close_BGI$,tick_volume_BGI$,spread_BGI$,real_volume_BGI$,open_CCM$,high_CCM$,...,EMA55_CCM$,EMA9_GOLD11,EMA21_GOLD11,EMA55_GOLD11,EMA9_BGI$,EMA21_BGI$,EMA55_BGI$,EMA9_IBOV,EMA21_IBOV,EMA55_IBOV
96,2022-06-02 09:00:00,313.94,314.62,313.07,313.75,44.0,1.0,14.0,121.02,122.52,...,122.110865,9.447520,9.432993,9.380451,312.964032,313.169828,313.822989,111438.765770,111480.083554,111455.854537
97,2022-06-02 09:15:00,313.17,313.17,312.73,312.73,4.0,1.0,4.0,122.10,122.76,...,122.120834,9.448016,9.434539,9.382935,312.917225,313.129843,313.783954,111423.012616,111469.166867,111452.431161
98,2022-06-02 09:30:00,313.17,313.17,312.73,312.73,0.0,1.0,0.0,122.45,122.66,...,122.120090,9.448413,9.435945,9.385330,312.879780,313.093494,313.746313,111410.410093,111459.242606,111449.130048
99,2022-06-02 09:45:00,313.41,313.41,312.78,312.78,5.0,1.0,5.0,121.98,121.98,...,122.100087,9.448730,9.437222,9.387640,312.859824,313.064995,313.711802,111400.328074,111450.220551,111445.946832
100,2022-06-02 10:00:00,312.83,313.99,312.83,313.56,9.0,1.0,9.0,121.64,121.78,...,122.083655,9.446984,9.437475,9.389510,312.999859,313.109995,313.706380,111577.062459,111526.018683,111475.877302


In [56]:
def pivot_data(df, target=None):
    """
    Pivot a DataFrame by hour for each day, excluding columns containing the target keyword.

    Parameters:
    df (pd.DataFrame): Input DataFrame with a 'time' column containing datetime values.
    target (str, optional): Keyword to exclude columns containing this value from the pivot. Default is None.

    Returns:
    pd.DataFrame: Pivoted DataFrame with columns for each hour of the day and variables suffixed with '_hhmm'.

    Example:
    >>> data = {
    ...     'time': ['2022-06-02 09:00:00', '2022-06-02 09:15:00', '2022-06-02 09:30:00'],
    ...     'open_BGI$': [313.94, 313.17, 312.85],
    ...     'close_BGI$': [313.75, 312.73, 313.00],
    ... }
    >>> df = pd.DataFrame(data)
    >>> df['time'] = pd.to_datetime(df['time'])
    >>> print(pivoted)
    date       open_BGI$_0900  open_BGI$_0915  open_BGI$_0930  close_BGI$_0900  close_BGI$_0915  close_BGI$_0930
    2022-06-02        313.94          313.17          312.85           313.75          312.73          313.00
    """
    df['date'] = pd.to_datetime(df['time']).dt.date
    df['hour'] = pd.to_datetime(df['time']).dt.strftime('%H%M')

    columns_to_pivot = [col for col in df.columns if col not in ['time', 'date', 'hour']]
    pivoted = df.pivot(index='date', columns='hour', values=columns_to_pivot)
    pivoted.columns = [f"{col}_{hour}" for col, hour in pivoted.columns]

    pivoted.reset_index(inplace=True)
    return pivoted


In [57]:
df_daily.columns

Index(['time', 'open_BGI$', 'high_BGI$', 'low_BGI$', 'close_BGI$',
       'tick_volume_BGI$', 'spread_BGI$', 'real_volume_BGI$', 'open_CCM$',
       'high_CCM$', 'low_CCM$', 'close_CCM$', 'tick_volume_CCM$',
       'spread_CCM$', 'real_volume_CCM$', 'open_GOLD11', 'high_GOLD11',
       'low_GOLD11', 'close_GOLD11', 'tick_volume_GOLD11', 'spread_GOLD11',
       'real_volume_GOLD11', 'open_IBOV', 'high_IBOV', 'low_IBOV',
       'close_IBOV', 'tick_volume_IBOV', 'spread_IBOV', 'real_volume_IBOV',
       'open_ICF$', 'high_ICF$', 'low_ICF$', 'close_ICF$', 'tick_volume_ICF$',
       'spread_ICF$', 'real_volume_ICF$', 'open_IVVB11', 'high_IVVB11',
       'low_IVVB11', 'close_IVVB11', 'tick_volume_IVVB11', 'spread_IVVB11',
       'real_volume_IVVB11', 'OBV_ICF$', 'OBV_IVVB11', 'OBV_CCM$',
       'OBV_GOLD11', 'OBV_BGI$', 'OBV_IBOV', 'RSI_ICF$', 'RSI_IVVB11',
       'RSI_CCM$', 'RSI_GOLD11', 'RSI_BGI$', 'RSI_IBOV', 'ATR_ICF$',
       'ATR_IVVB11', 'ATR_CCM$', 'ATR_GOLD11', 'ATR_BGI$', 'ATR_IBO

In [58]:
df_daily =  pivot_data(df_daily.copy())


In [59]:
df_daily.columns

Index(['date', 'open_BGI$_0900', 'open_BGI$_0915', 'open_BGI$_0930',
       'open_BGI$_0945', 'open_BGI$_1000', 'open_BGI$_1015', 'open_BGI$_1030',
       'open_BGI$_1045', 'open_BGI$_1100',
       ...
       'EMA55_IBOV_1530', 'EMA55_IBOV_1545', 'EMA55_IBOV_1600',
       'EMA55_IBOV_1615', 'EMA55_IBOV_1630', 'EMA55_IBOV_1645',
       'EMA55_IBOV_1700', 'EMA55_IBOV_1715', 'EMA55_IBOV_1730',
       'EMA55_IBOV_1745'],
      dtype='object', length=3025)

In [60]:
df_daily_target.columns

Index(['open_BGI$', 'close_BGI$', 'close_price_target', 'open_price_target',
       'behavior_target'],
      dtype='object')

In [ ]:
df_daily = df_daily.merge(df_daily_target[['day', 'close_price_target', 'open_price_target', 'behavior_target']],
    left_on='date', right_on='day',
    how='inner'
)

In [65]:
save_table(df_daily.head(6), title = 'Exemplo do Target diário para o fechamento, abertura e comportamento do mercado')
save_table(df_timestamp.head(6), title = 'Exemplo do Target timestamp para o fechamento, abertura e comportamento do mercado')

Tabela saved as CSV: ../results/tables/csv\Tabela_2_Exemplo do Target diário para o fechamento, abertura e comportamento do mercado.csv
Tabela saved as CSV: ../results/tables/csv\Tabela_3_Exemplo do Target timestamp para o fechamento, abertura e comportamento do mercado.csv


In [66]:
os.makedirs(output_dir, exist_ok=True)
df_daily.to_parquet(f'{output_dir}df_daily.parquet')
df_timestamp.to_parquet(f'{output_dir}df_timestamp.parquet')